<h1 style="background-color:#3b5998;font-family:newtimeroman;font-size:300%;text-align:center;color:#FFFFFF">Çoklu Doğrusallık Varsayımı:
</h1>

In [1]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("car_price.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   wheelbase         205 non-null    float64
 3   carlength         205 non-null    float64
 4   carwidth          205 non-null    float64
 5   carheight         205 non-null    float64
 6   curbweight        205 non-null    int64  
 7   enginesize        205 non-null    int64  
 8   boreratio         205 non-null    float64
 9   stroke            205 non-null    float64
 10  compressionratio  205 non-null    float64
 11  horsepower        205 non-null    int64  
 12  peakrpm           205 non-null    int64  
 13  citympg           205 non-null    int64  
 14  highwaympg        205 non-null    int64  
 15  price             205 non-null    float64
dtypes: float64(8), int64(8)
memory usage: 25.8 K

###### Veri seti 256 satır 16 sütundan oluştuğu gözlemlendi

In [4]:
df.shape

(205, 16)

In [5]:
df.head()

,Unnamed: 0,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,0,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,13495.0
1,1,3,88.6,168.8,64.1,48.8,2548,130,3.47,2.68,9.0,111,5000,21,27,16500.0
2,2,1,94.5,171.2,65.5,52.4,2823,152,2.68,3.47,9.0,154,5000,19,26,16500.0
3,3,2,99.8,176.6,66.2,54.3,2337,109,3.19,3.40,10.0,102,5500,24,30,13950.0
4,4,2,99.4,176.6,66.4,54.3,2824,136,3.19,3.40,8.0,115,5500,18,22,17450.0


In [6]:
df.isnull().sum()

Unnamed: 0          0
symboling           0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginesize          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [7]:
df.drop("Unnamed: 0",axis=1,inplace=True)

In [8]:
num_cols = [col for col in df.columns if df[col].dtypes in ("int64","float64")]
num_cols

['symboling',
 'wheelbase',
 'carlength',
 'carwidth',
 'carheight',
 'curbweight',
 'enginesize',
 'boreratio',
 'stroke',
 'compressionratio',
 'horsepower',
 'peakrpm',
 'citympg',
 'highwaympg',
 'price']

In [9]:
X = df.drop("price",axis=1)

In [10]:
y = df["price"]

<h1 style="background-color:#FF00F7;font-family:newtimeroman;font-size:150%;text-align:left;color:#FFFFFF">1)VIF (Variance Inflation Factor):
</h1>

###### VIF değerleri hesaplandı.

VIF, her bağımsız değişken için ayrı ayrı hesaplanır.

$\text{VIF}_i = \frac{1}{1 - R_i^2}$

$\text{VIF}_i$ = İncelenen bağımsız değişkenin VIF değeri

$ R_i^2 $  = İncelenen bağımsız değişkenin diğer bağımsız değişkenlerle olan determinasyon katsayısıdır.

In [11]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data["VIF"]

0        2.540920
1     2254.716977
2     1893.738263
3     2828.144983
4      929.458469
5      403.471933
6       68.872234
7      291.357019
8      126.412487
9       15.834999
10      66.166398
11     219.618733
12     437.334307
13     511.133157
Name: VIF, dtype: float64

###### Çoklu doğrusallık problemı olan yani VIF değeri 10'dan büyük olan değerler gözlemlendi.

In [12]:
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
3,carwidth,2828.144983
1,wheelbase,2254.716977
2,carlength,1893.738263
4,carheight,929.458469
13,highwaympg,511.133157
12,citympg,437.334307
5,curbweight,403.471933
7,boreratio,291.357019
11,peakrpm,219.618733
8,stroke,126.412487


<h1 style="background-color:#FF00F7;font-family:newtimeroman;font-size:150%;text-align:left;color:#FFFFFF">2)Tolerans:
</h1>

###### ${1 / VIF_i}$

In [13]:
vif_values = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
tolerance_values = [1 / vif for vif in vif_values]
tolerance_data = pd.DataFrame({'Variable': X.columns, 'Tolerance': tolerance_values})
sorted_tolerance_data = tolerance_data.sort_values(by='Tolerance', ascending=False)
sorted_tolerance_data = pd.DataFrame(sorted_tolerance_data)
sorted_tolerance_data

,Variable,Tolerance
0,symboling,0.393558
9,compressionratio,0.063151
10,horsepower,0.015113
6,enginesize,0.014520
8,stroke,0.007911
11,peakrpm,0.004553
7,boreratio,0.003432
5,curbweight,0.002478
12,citympg,0.002287
13,highwaympg,0.001956


<h1 style="background-color:#FF00F7;font-family:newtimeroman;font-size:150%;text-align:left;color:#FFFFFF">3)Korelasyon Matrisi İncelemesi:
</h1>

In [14]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)

print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    print(pair)


Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength')
('wheelbase', 'carwidth')
('wheelbase', 'curbweight')
('carlength', 'carwidth')
('carlength', 'curbweight')
('carlength', 'highwaympg')
('carwidth', 'curbweight')
('carwidth', 'enginesize')
('curbweight', 'enginesize')
('curbweight', 'horsepower')
('curbweight', 'citympg')
('curbweight', 'highwaympg')
('enginesize', 'horsepower')
('horsepower', 'citympg')
('horsepower', 'highwaympg')
('citympg', 'highwaympg')


##### Orijinal veri seti için model özeti

In [15]:
X = sm.add_constant(df.drop("price",axis=1))
y = df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        00:41:17   Log-Likelihood:                -1936.7
No. Observations:                 205   AIC:                             3903.
Df Residuals:                     190   BIC:                             3953.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -5.165e+04   1.57e+04     -3.299      0.001   -8.25e+04   -2.08e+04
symboling          285.8829    243.335      1.175      0.242    -194.101     765.867
wheelbase          167.6990    107.450      1.561      0.120     -44.250     379.648
carlength          -94.8179     55.502     -1.708      0.089    -204.297      14.661
carwidth           466.6185    247.995      1.882      0.061     -22.559     955.796
carheight          194.7522    138.223      1.409      0.160     -77.897     467.402
curbweight           1.8776      1.736      1.082      0.281      -1.546       5.301
enginesize         116.7820     13.831      8.443      0.000      89.500     144.064
boreratio         -984.4276   1194.709     -0.824      0.411   -3341.025    1372.169
stroke           -3056.1620    778.046     -3.928      0.000   -4590.881   -1521.443
compressionratio   286.4752     83.425      3.434      0.001     121.918     451.033
horsepower          32.5014     16.264      1.998      0.047       0.420      64.583
peakrpm              2.3582      0.670      3.518      0.001       1.036       3.680
citympg           -286.9397    179.856     -1.595      0.112    -641.710      67.831
highwaympg         191.3036    159.902      1.196      0.233    -124.108     506.716
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                     4.05e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.05e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<h1 style="background-color:#FFA500;font-family:newtimeroman;font-size:200%;text-align:left;color:#FFFFFF">Çözüm Yöntemleri:
</h1>


#### VIF değerleri 10'dan büyük olan değişkenleri modelden çıkartıp tekrar model kurup sonuclarını inceleyelim.


In [16]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenler kaldırdıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [17]:
X = sm.add_constant(X)
y = y
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.307
Date:                Wed, 24 Jan 2024   Prob (F-statistic):              0.254
Time:                        00:41:17   Log-Likelihood:                -2131.8
No. Observations:                 205   AIC:                             4268.
Df Residuals:                     203   BIC:                             4274.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        1.37e+04    671.577     20.407      0.000    1.24e+04     1.5e+04
symboling   -513.0737    448.814     -1.143      0.254   -1398.009     371.862
==============================================================================
Omnibus:                       77.789   Durbin-Watson:                   0.444
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              183.196
Skew:                           1.769   Prob(JB):                     1.66e-40
Kurtosis:                       5.989   Cond. No.                         2.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">Logaritmik Dönüşüm
</h1>

In [18]:
log_df = df.copy()

In [19]:
for col in num_cols:
    if df[col].min() > 0: 
        log_df[col] = np.log(df[col])

In [20]:
log_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,4.484132,5.128715,4.160444,3.887730,7.843064,4.867534,1.244155,0.985817,2.197225,4.709530,8.517193,3.044522,3.295837,9.510075
1,3,4.484132,5.128715,4.160444,3.887730,7.843064,4.867534,1.244155,0.985817,2.197225,4.709530,8.517193,3.044522,3.295837,9.711116
2,1,4.548600,5.142832,4.182050,3.958907,7.945555,5.023881,0.985817,1.244155,2.197225,5.036953,8.517193,2.944439,3.258097,9.711116
3,2,4.603168,5.173887,4.192680,3.994524,7.756623,4.691348,1.160021,1.223775,2.302585,4.624973,8.612503,3.178054,3.401197,9.543235
4,2,4.599152,5.173887,4.195697,3.994524,7.945910,4.912655,1.160021,1.223775,2.079442,4.744932,8.612503,2.890372,3.091042,9.767095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,4.692265,5.240688,4.232656,4.016383,7.990238,4.948760,1.329724,1.147402,2.251292,4.736198,8.594154,3.135494,3.332205,9.731809
201,-1,4.692265,5.240688,4.231204,4.016383,8.022569,4.948760,1.329724,1.147402,2.163323,5.075174,8.575462,2.944439,3.218876,9.854560
202,-1,4.692265,5.240688,4.232656,4.016383,8.010360,5.153292,1.275363,1.054312,2.174752,4.897840,8.612503,2.890372,3.135494,9.975110
203,-1,4.692265,5.240688,4.232656,4.016383,8.076205,4.976734,1.101940,1.223775,3.135494,4.663439,8.476371,3.258097,3.295837,10.019936


###### Logaritmik dönüşümü yapıldıktan sonra model $R^2$ değerinin orijinal veri  $R^2$  'ne göre arttığı gözlemlendi.

In [21]:
X = sm.add_constant(log_df.drop("price",axis=1))
y = log_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.883
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                     102.5
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           1.68e-80
Time:                        00:41:17   Log-Likelihood:                 70.169
No. Observations:                 205   AIC:                            -110.3
Df Residuals:                     190   BIC:                            -60.49
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -8.8731      3.928     -2.259      0.025     -16.620      -1.126
symboling            0.0092      0.014      0.661      0.509      -0.018       0.037
wheelbase            0.8300      0.597      1.390      0.166      -0.347       2.007
carlength           -0.5232      0.528     -0.991      0.323      -1.565       0.519
carwidth             1.5374      0.920      1.672      0.096      -0.277       3.351
carheight            0.4133      0.417      0.992      0.322      -0.408       1.235
curbweight           0.4369      0.271      1.611      0.109      -0.098       0.972
enginesize           0.4715      0.116      4.069      0.000       0.243       0.700
boreratio           -0.2384      0.224     -1.063      0.289      -0.681       0.204
stroke              -0.3823      0.134     -2.847      0.005      -0.647      -0.117
compressionratio     0.3855      0.069      5.559      0.000       0.249       0.522
horsepower           0.3984      0.122      3.253      0.001       0.157       0.640
peakrpm              0.3215      0.200      1.609      0.109      -0.073       0.716
citympg             -0.8750      0.254     -3.441      0.001      -1.377      -0.373
highwaympg           0.4196      0.247      1.697      0.091      -0.068       0.907
==============================================================================
Omnibus:                        2.238   Durbin-Watson:                   0.968
Prob(Omnibus):                  0.327   Jarque-Bera (JB):                1.925
Skew:                           0.228   Prob(JB):                        0.382
Kurtosis:                       3.132   Cond. No.                     5.42e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.42e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.79
('wheelbase', 'curbweight'): 0.76
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.89
('carlength', 'enginesize'): 0.73
('carlength', 'highwaympg'): -0.71
('carwidth', 'curbweight'): 0.86
('carwidth', 'enginesize'): 0.76
('carwidth', 'highwaympg'): -0.71
('curbweight', 'enginesize'): 0.86
('curbweight', 'horsepower'): 0.79
('curbweight', 'citympg'): -0.80
('curbweight', 'highwaympg'): -0.84
('enginesize', 'horsepower'): 0.81
('enginesize', 'citympg'): -0.72
('enginesize', 'highwaympg'): -0.74
('horsepower', 'citympg'): -0.90
('horsepower', 'highwaympg'): -0.86
('citympg', 'highwaympg'): 0.96


 ###### Çoklu bağlantı problemine sahip değişken sayısının 5'e düştüğü gözlemlendi

In [23]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
0,const,99261.475449
13,citympg,27.337476
14,highwaympg,19.697737
6,curbweight,18.462032
11,horsepower,11.426061


In [24]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenler kaldırdıktan sonra $R^2$ değeri logaritmik dönüşümü ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [25]:
X = sm.add_constant(X)
y = log_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     94.06
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.71e-69
Time:                        00:41:17   Log-Likelihood:                 31.190
No. Observations:                 205   AIC:                            -40.38
Df Residuals:                     194   BIC:                            -3.828
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const              -27.6284      3.556     -7.770      0.000     -34.641     -20.615
symboling            0.0354      0.016      2.221      0.028       0.004       0.067
wheelbase            0.5065      0.685      0.740      0.460      -0.844       1.857
carlength            1.2509      0.556      2.249      0.026       0.154       2.348
carwidth             3.4078      1.069      3.188      0.002       1.300       5.516
carheight           -0.1474      0.488     -0.302      0.763      -1.110       0.815
enginesize           1.0272      0.098     10.469      0.000       0.834       1.221
boreratio            0.2717      0.258      1.054      0.293      -0.237       0.780
stroke              -0.4834      0.159     -3.040      0.003      -0.797      -0.170
compressionratio     0.1819      0.062      2.916      0.004       0.059       0.305
peakrpm              1.1002      0.199      5.538      0.000       0.708       1.492
==============================================================================
Omnibus:                        7.835   Durbin-Watson:                   0.955
Prob(Omnibus):                  0.020   Jarque-Bera (JB):                8.061
Skew:                           0.485   Prob(JB):                       0.0178
Kurtosis:                       2.964   Cond. No.                     3.33e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.33e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [26]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.79
('carlength', 'carwidth'): 0.84
('carlength', 'enginesize'): 0.73
('carwidth', 'enginesize'): 0.76


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">Karekök Dönüşüm:
</h1>

In [27]:
sqrt_df = df.copy()

In [28]:
for col in num_cols:
    if df[col].min() > 0:
        sqrt_df[col] = np.sqrt(df[col])   

In [29]:
sqrt_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,9.412757,12.992305,8.006248,6.985700,50.477718,11.401754,1.862794,1.637071,3.000000,10.535654,70.710678,4.582576,5.196152,116.167982
1,3,9.412757,12.992305,8.006248,6.985700,50.477718,11.401754,1.862794,1.637071,3.000000,10.535654,70.710678,4.582576,5.196152,128.452326
2,1,9.721111,13.084342,8.093207,7.238784,53.131911,12.328828,1.637071,1.862794,3.000000,12.409674,70.710678,4.358899,5.099020,128.452326
3,2,9.989995,13.289093,8.136338,7.368853,48.342528,10.440307,1.786057,1.843909,3.162278,10.099505,74.161985,4.898979,5.477226,118.110118
4,2,9.969955,13.289093,8.148620,7.368853,53.141321,11.661904,1.786057,1.843909,2.828427,10.723805,74.161985,4.242641,4.690416,132.098448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,10.445095,13.740451,8.300602,7.449832,54.332311,11.874342,1.944222,1.774824,3.082207,10.677078,73.484692,4.795832,5.291503,129.788289
201,-1,10.445095,13.740451,8.294577,7.449832,55.217751,11.874342,1.944222,1.774824,2.949576,12.649111,72.801099,4.358899,5.000000,138.003623
202,-1,10.445095,13.740451,8.300602,7.449832,54.881691,13.152946,1.892089,1.694107,2.966479,11.575837,74.161985,4.242641,4.795832,146.577624
203,-1,10.445095,13.740451,8.300602,7.449832,56.718604,12.041595,1.734935,1.843909,4.795832,10.295630,69.282032,5.099020,5.196152,149.899967


###### Karekök dönüşümü yapıldıktan sonra model $R^2$ değerinin orijinal veri setine göre arttığı gözlemlendi.

In [30]:
X = sm.add_constant(sqrt_df.drop("price",axis=1))
y = sqrt_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.869
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     89.74
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           9.85e-76
Time:                        00:41:17   Log-Likelihood:                -783.31
No. Observations:                 205   AIC:                             1597.
Df Residuals:                     190   BIC:                             1646.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             -334.0287    113.840     -2.934      0.004    -558.582    -109.476
symboling            0.8777      0.886      0.990      0.323      -0.871       2.626
wheelbase           13.4281      7.689      1.746      0.082      -1.739      28.596
carlength           -8.3651      5.214     -1.604      0.110     -18.651       1.921
carwidth            27.1855     14.542      1.869      0.063      -1.499      55.870
carheight            8.0505      7.296      1.103      0.271      -6.340      22.441
curbweight           0.8644      0.671      1.288      0.199      -0.460       2.189
enginesize           8.1204      1.267      6.408      0.000       5.621      10.620
boreratio          -18.0107     15.739     -1.144      0.254     -49.055      13.034
stroke             -36.8562      9.850     -3.742      0.000     -56.286     -17.426
compressionratio    10.6282      2.364      4.496      0.000       5.966      15.291
horsepower           3.7182      1.417      2.623      0.009       0.923       6.514
peakrpm              0.9633      0.355      2.717      0.007       0.264       1.663
citympg            -16.0192      6.628     -2.417      0.017     -29.094      -2.944
highwaympg           8.5556      6.136      1.394      0.165      -3.549      20.660
==============================================================================
Omnibus:                        7.462   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.024   Jarque-Bera (JB):                9.256
Skew:                           0.272   Prob(JB):                      0.00978
Kurtosis:                       3.888   Cond. No.                     1.30e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.3e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [31]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.79
('wheelbase', 'curbweight'): 0.77
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.88
('carlength', 'enginesize'): 0.71
('carlength', 'highwaympg'): -0.71
('carwidth', 'curbweight'): 0.87
('carwidth', 'enginesize'): 0.75
('curbweight', 'enginesize'): 0.86
('curbweight', 'horsepower'): 0.78
('curbweight', 'citympg'): -0.79
('curbweight', 'highwaympg'): -0.82
('enginesize', 'horsepower'): 0.82
('enginesize', 'highwaympg'): -0.72
('horsepower', 'citympg'): -0.86
('horsepower', 'highwaympg'): -0.83
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 5'e düştüğü gözlemlendi

In [32]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
0,const,20180.541769
13,citympg,28.016474
14,highwaympg,22.127788
6,curbweight,17.798281
11,horsepower,10.205384


In [33]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenler veri setinden kaldırdıktan sonra $R^2$ değeri karekök dönüşümü ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [34]:
X = sm.add_constant(X)
y = sqrt_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     100.7
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           3.14e-71
Time:                        00:41:17   Log-Likelihood:                -804.47
No. Observations:                 205   AIC:                             1631.
Df Residuals:                     194   BIC:                             1667.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             -680.6918    103.152     -6.599      0.000    -884.134    -477.249
symboling            1.8664      0.936      1.993      0.048       0.019       3.713
wheelbase            6.9959      7.991      0.875      0.382      -8.765      22.757
carlength            4.3897      4.997      0.878      0.381      -5.465      14.245
carwidth            49.1134     15.391      3.191      0.002      18.758      79.469
carheight            2.9587      7.905      0.374      0.709     -12.633      18.550
enginesize          12.4242      0.937     13.255      0.000      10.576      14.273
boreratio           10.7060     16.537      0.647      0.518     -21.910      43.322
stroke             -40.2840     10.732     -3.754      0.000     -61.451     -19.117
compressionratio     6.3605      1.990      3.196      0.002       2.435      10.286
peakrpm              1.9191      0.326      5.878      0.000       1.275       2.563
==============================================================================
Omnibus:                       10.296   Durbin-Watson:                   0.875
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               10.376
Skew:                           0.521   Prob(JB):                      0.00558
Kurtosis:                       3.360   Cond. No.                     8.90e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.9e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.79
('carlength', 'carwidth'): 0.84
('carlength', 'enginesize'): 0.71
('carwidth', 'enginesize'): 0.75


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">Küp Dönüşüm
</h1>

In [36]:
cube_df = df.copy()

In [37]:
for col in num_cols:
    cube_df[col] = df[col] ** 3

In [38]:
cube_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,27,695506.456,4809692.672,263374.721,116214.272,16542390592,2197000,41.781923,19.248832,729.000,1367631,125000000000,9261,19683,2.457642e+12
1,27,695506.456,4809692.672,263374.721,116214.272,16542390592,2197000,41.781923,19.248832,729.000,1367631,125000000000,9261,19683,4.492125e+12
2,1,843908.625,5017776.128,281011.375,143877.824,22497415767,3511808,19.248832,41.781923,729.000,3652264,125000000000,6859,17576,4.492125e+12
3,8,994011.992,5507723.096,290117.528,160103.007,12763686753,1295029,32.461759,39.304000,1000.000,1061208,166375000000,13824,27000,2.714705e+12
4,8,982107.784,5507723.096,292754.944,160103.007,22521332224,2515456,32.461759,39.304000,512.000,1520875,166375000000,5832,10648,5.313569e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,1298596.571,6729859.072,327082.769,170953.875,25724625408,2803221,54.010152,31.255875,857.375,1481544,157464000000,12167,21952,4.779837e+12
201,-1,1298596.571,6729859.072,325660.672,170953.875,28344726649,2803221,54.010152,31.255875,658.503,4096000,148877000000,6859,15625,6.907851e+12
202,-1,1298596.571,6729859.072,327082.769,170953.875,27325297728,5177717,45.882712,23.639903,681.472,2406104,166375000000,5832,12167,9.917588e+12
203,-1,1298596.571,6729859.072,327082.769,170953.875,33293019313,3048625,27.270901,39.304000,12167.000,1191016,110592000000,17576,19683,1.134512e+13


###### Küp dönüşümü yapıldıktan sonraki $R^2$ değeri orijinal veri setinin $R^2$ değerine göre düşük çıktığı gözlemlendi.

In [39]:
X = sm.add_constant(cube_df.drop("price",axis=1))
y = cube_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.754
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     41.59
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           3.30e-50
Time:                        00:41:17   Log-Likelihood:                -6337.2
No. Observations:                 205   AIC:                         1.270e+04
Df Residuals:                     190   BIC:                         1.275e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -3.333e+13   1.06e+13     -3.134      0.002   -5.43e+13   -1.23e+13
symboling         5.723e+10    6.5e+10      0.880      0.380    -7.1e+10    1.85e+11
wheelbase        -2.647e+06   7.39e+06     -0.358      0.721   -1.72e+07    1.19e+07
carlength        -8.611e+05   1.27e+06     -0.679      0.498   -3.36e+06    1.64e+06
carwidth          6.185e+07   3.88e+07      1.592      0.113   -1.48e+07    1.38e+08
carheight         5.903e+07   3.33e+07      1.770      0.078   -6.75e+06    1.25e+08
curbweight         120.6709    121.662      0.992      0.323    -119.310     360.652
enginesize         2.35e+06   2.12e+05     11.096      0.000    1.93e+06    2.77e+06
boreratio         1.049e+11   7.39e+10      1.418      0.158    -4.1e+10    2.51e+11
stroke           -7.596e+10   5.43e+10     -1.398      0.164   -1.83e+11    3.12e+10
compressionratio  1.393e+08   2.04e+08      0.683      0.495   -2.63e+08    5.41e+08
horsepower        2.163e+04   3.31e+05      0.065      0.948   -6.31e+05    6.74e+05
peakrpm             50.4684     15.448      3.267      0.001      19.997      80.940
citympg          -6.285e+07   1.37e+08     -0.458      0.647   -3.33e+08    2.08e+08
highwaympg        8.347e+07   9.81e+07      0.851      0.396    -1.1e+08    2.77e+08
==============================================================================
Omnibus:                      134.133   Durbin-Watson:                   1.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4331.518
Skew:                           1.892   Prob(JB):                         0.00
Kurtosis:                      25.199   Cond. No.                     3.28e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.28e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [40]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.89
('wheelbase', 'carwidth'): 0.79
('wheelbase', 'curbweight'): 0.78
('carlength', 'carwidth'): 0.85
('carlength', 'curbweight'): 0.85
('carwidth', 'curbweight'): 0.85
('curbweight', 'enginesize'): 0.77
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 4'e düştüğü gözlemlendi

In [41]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
0,const,517.387157
14,highwaympg,25.313982
13,citympg,23.450753
6,curbweight,10.246216


In [42]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri küp dönüşümü ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [43]:
X = sm.add_constant(X)
y = cube_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.751
Model:                            OLS   Adj. R-squared:                  0.737
Method:                 Least Squares   F-statistic:                     52.95
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           2.69e-52
Time:                        00:41:18   Log-Likelihood:                -6338.4
No. Observations:                 205   AIC:                         1.270e+04
Df Residuals:                     193   BIC:                         1.274e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -3.097e+13   9.04e+12     -3.427      0.001   -4.88e+13   -1.31e+13
symboling         5.197e+10    6.3e+10      0.825      0.410   -7.22e+10    1.76e+11
wheelbase        -1.261e+06   7.09e+06     -0.178      0.859   -1.52e+07    1.27e+07
carlength        -8.958e+05   1.15e+06     -0.781      0.436   -3.16e+06    1.37e+06
carwidth          6.419e+07   3.83e+07      1.678      0.095   -1.13e+07     1.4e+08
carheight         5.956e+07   3.33e+07      1.791      0.075   -6.04e+06    1.25e+08
enginesize        2.456e+06   1.85e+05     13.296      0.000    2.09e+06    2.82e+06
boreratio         8.844e+10    7.2e+10      1.228      0.221   -5.36e+10     2.3e+11
stroke           -6.856e+10    5.3e+10     -1.294      0.197   -1.73e+11     3.6e+10
compressionratio  2.566e+08   1.83e+08      1.400      0.163   -1.05e+08    6.18e+08
horsepower         1.09e+05   3.14e+05      0.347      0.729    -5.1e+05    7.28e+05
peakrpm             45.4816     15.047      3.023      0.003      15.805      75.158
==============================================================================
Omnibus:                      133.183   Durbin-Watson:                   1.259
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4574.030
Skew:                           1.848   Prob(JB):                         0.00
Kurtosis:                      25.844   Cond. No.                     2.78e+12
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.78e+12. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [44]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.89
('wheelbase', 'carwidth'): 0.79
('carlength', 'carwidth'): 0.85


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">MİN MAX SCALER
</h1>

In [45]:
minmax_df = df.copy()

In [46]:
scaler = MinMaxScaler()

for col in num_cols:
    minmax_df[col] = scaler.fit_transform(df[[col]])

In [47]:
minmax_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1.0,0.058309,0.413433,0.316667,0.083333,0.411171,0.260377,0.664286,0.290476,0.12500,0.262500,0.346939,0.222222,0.289474,0.207959
1,1.0,0.058309,0.413433,0.316667,0.083333,0.411171,0.260377,0.664286,0.290476,0.12500,0.262500,0.346939,0.222222,0.289474,0.282558
2,0.6,0.230321,0.449254,0.433333,0.383333,0.517843,0.343396,0.100000,0.666667,0.12500,0.441667,0.346939,0.166667,0.263158,0.282558
3,0.8,0.384840,0.529851,0.491667,0.541667,0.329325,0.181132,0.464286,0.633333,0.18750,0.225000,0.551020,0.305556,0.368421,0.219254
4,0.8,0.373178,0.529851,0.508333,0.541667,0.518231,0.283019,0.464286,0.633333,0.06250,0.279167,0.551020,0.138889,0.157895,0.306142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,0.2,0.655977,0.711940,0.716667,0.641667,0.567882,0.301887,0.885714,0.514286,0.15625,0.275000,0.510204,0.277778,0.315789,0.291123
201,0.2,0.655977,0.711940,0.708333,0.641667,0.605508,0.301887,0.885714,0.514286,0.10625,0.466667,0.469388,0.166667,0.236842,0.345738
202,0.2,0.655977,0.711940,0.716667,0.641667,0.591156,0.422642,0.742857,0.380952,0.11250,0.358333,0.551020,0.138889,0.184211,0.406311
203,0.2,0.655977,0.711940,0.716667,0.641667,0.670675,0.316981,0.335714,0.633333,1.00000,0.241667,0.265306,0.361111,0.289474,0.430763


###### MinMax dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değeriyle aynı çıktığı gözlemlendi.

In [48]:
X = sm.add_constant(minmax_df.drop("price",axis=1))
y = minmax_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        00:41:18   Log-Likelihood:                 237.02
No. Observations:                 205   AIC:                            -444.0
Df Residuals:                     190   BIC:                            -394.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1236      0.075     -1.650      0.101      -0.271       0.024
symboling            0.0355      0.030      1.175      0.242      -0.024       0.095
wheelbase            0.1428      0.091      1.561      0.120      -0.038       0.323
carlength           -0.1577      0.092     -1.708      0.089      -0.340       0.024
carwidth             0.1390      0.074      1.882      0.061      -0.007       0.285
carheight            0.0580      0.041      1.409      0.160      -0.023       0.139
curbweight           0.1202      0.111      1.082      0.281      -0.099       0.339
enginesize           0.7683      0.091      8.443      0.000       0.589       0.948
boreratio           -0.0342      0.042     -0.824      0.411      -0.116       0.048
stroke              -0.1593      0.041     -3.928      0.000      -0.239      -0.079
compressionratio     0.1138      0.033      3.434      0.001       0.048       0.179
horsepower           0.1936      0.097      1.998      0.047       0.003       0.385
peakrpm              0.1434      0.041      3.518      0.001       0.063       0.224
citympg             -0.2564      0.161     -1.595      0.112      -0.573       0.061
highwaympg           0.1805      0.151      1.196      0.233      -0.117       0.478
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                         75.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('wheelbase', 'curbweight'): 0.78
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.88
('carlength', 'highwaympg'): -0.70
('carwidth', 'curbweight'): 0.87
('carwidth', 'enginesize'): 0.74
('curbweight', 'enginesize'): 0.85
('curbweight', 'horsepower'): 0.75
('curbweight', 'citympg'): -0.76
('curbweight', 'highwaympg'): -0.80
('enginesize', 'horsepower'): 0.81
('horsepower', 'citympg'): -0.80
('horsepower', 'highwaympg'): -0.77
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 4'e düştüğü gözlemlendi

In [50]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
0,const,183.797764
13,citympg,27.824477
14,highwaympg,24.369050
6,curbweight,16.413566


In [51]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()

In [52]:
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenler kaldırdıktan sonra $R^2$ değeri MinMax ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [53]:
X = sm.add_constant(X)
y = minmax_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     98.03
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           8.80e-73
Time:                        00:41:18   Log-Likelihood:                 234.50
No. Observations:                 205   AIC:                            -445.0
Df Residuals:                     193   BIC:                            -405.1
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.1825      0.047     -3.921      0.000      -0.274      -0.091
symboling            0.0487      0.029      1.654      0.100      -0.009       0.107
wheelbase            0.1475      0.089      1.665      0.098      -0.027       0.322
carlength           -0.0632      0.081     -0.776      0.439      -0.224       0.097
carwidth             0.1533      0.073      2.090      0.038       0.009       0.298
carheight            0.0580      0.041      1.408      0.161      -0.023       0.139
enginesize           0.7643      0.084      9.138      0.000       0.599       0.929
boreratio           -0.0239      0.041     -0.579      0.563      -0.105       0.058
stroke              -0.1523      0.040     -3.761      0.000      -0.232      -0.072
compressionratio     0.1030      0.027      3.828      0.000       0.050       0.156
horsepower           0.2996      0.084      3.581      0.000       0.135       0.465
peakrpm              0.1382      0.040      3.440      0.001       0.059       0.217
==============================================================================
Omnibus:                       23.011   Durbin-Watson:                   0.902
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.223
Skew:                           0.410   Prob(JB):                     6.87e-15
Kurtosis:                       5.639   Cond. No.                         38.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('carlength', 'carwidth'): 0.84
('carwidth', 'enginesize'): 0.74
('enginesize', 'horsepower'): 0.81


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">Standard Scaler
</h1>

In [55]:
standard_df = df.copy()

In [56]:
standard_scaler = StandardScaler()
for col in num_cols:
    standard_df[col] = standard_scaler.fit_transform(df[[col]])


In [57]:
standard_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1.743470,-1.690772,-0.426521,-0.844782,-2.020417,-0.014566,0.074449,0.519071,-1.839377,-0.288349,0.174483,-0.262960,-0.646553,-0.546059,0.027391
1,1.743470,-1.690772,-0.426521,-0.844782,-2.020417,-0.014566,0.074449,0.519071,-1.839377,-0.288349,0.174483,-0.262960,-0.646553,-0.546059,0.404461
2,0.133509,-0.708596,-0.231513,-0.190566,-0.543527,0.514882,0.604046,-2.404880,0.685946,-0.288349,1.264536,-0.262960,-0.953012,-0.691627,0.404461
3,0.938490,0.173698,0.207256,0.136542,0.235942,-0.420797,-0.431076,-0.517266,0.462183,-0.035973,-0.053668,0.787855,-0.186865,-0.109354,0.084485
4,0.938490,0.107110,0.207256,0.230001,0.235942,0.516807,0.218885,-0.517266,0.462183,-0.540725,0.275883,0.787855,-1.106241,-1.273900,0.523668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1.476452,1.721873,1.198549,1.398245,0.728239,0.763241,0.339248,1.666445,-0.336970,-0.162161,0.250533,0.577692,-0.340094,-0.400490,0.447752
201,-1.476452,1.721873,1.198549,1.351515,0.728239,0.949992,0.339248,1.666445,-0.336970,-0.364062,1.416637,0.367529,-0.953012,-0.837195,0.723810
202,-1.476452,1.721873,1.198549,1.398245,0.728239,0.878757,1.109571,0.926204,-1.232021,-0.338824,0.757535,0.787855,-1.106241,-1.128332,1.029983
203,-1.476452,1.721873,1.198549,1.398245,0.728239,1.273437,0.435538,-1.183483,0.462183,3.244916,0.047732,-0.683286,0.119594,-0.546059,1.153582


###### StandardScale  dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değeriyle aynı çıktığı gözlemlendi.

In [58]:
X = sm.add_constant(standard_df.drop("price",axis=1))
y = standard_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        00:41:18   Log-Likelihood:                -95.143
No. Observations:                 205   AIC:                             220.3
Df Residuals:                     190   BIC:                             270.1
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.769e-16      0.028   6.34e-15      1.000      -0.055       0.055
symboling            0.0446      0.038      1.175      0.242      -0.030       0.119
wheelbase            0.1264      0.081      1.561      0.120      -0.033       0.286
carlength           -0.1464      0.086     -1.708      0.089      -0.315       0.023
carwidth             0.1253      0.067      1.882      0.061      -0.006       0.257
carheight            0.0596      0.042      1.409      0.160      -0.024       0.143
curbweight           0.1224      0.113      1.082      0.281      -0.101       0.346
enginesize           0.6087      0.072      8.443      0.000       0.467       0.751
boreratio           -0.0334      0.041     -0.824      0.411      -0.113       0.047
stroke              -0.1200      0.031     -3.928      0.000      -0.180      -0.060
compressionratio     0.1424      0.041      3.434      0.001       0.061       0.224
horsepower           0.1609      0.081      1.998      0.047       0.002       0.320
peakrpm              0.1408      0.040      3.518      0.001       0.062       0.220
citympg             -0.2350      0.147     -1.595      0.112      -0.526       0.056
highwaympg           0.1649      0.138      1.196      0.233      -0.107       0.437
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                         18.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('wheelbase', 'curbweight'): 0.78
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.88
('carlength', 'highwaympg'): -0.70
('carwidth', 'curbweight'): 0.87
('carwidth', 'enginesize'): 0.74
('curbweight', 'enginesize'): 0.85
('curbweight', 'horsepower'): 0.75
('curbweight', 'citympg'): -0.76
('curbweight', 'highwaympg'): -0.80
('enginesize', 'horsepower'): 0.81
('horsepower', 'citympg'): -0.80
('horsepower', 'highwaympg'): -0.77
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 3'e düştüğü gözlemlendi.

In [60]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
13,citympg,27.824477
14,highwaympg,24.369050
6,curbweight,16.413566


In [61]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri StandardScale ile dönüştürülmüş veri setinin $R^2$ değeri ile aynı çıktığı gözlemlendi.

In [62]:
X = sm.add_constant(X)
y = minmax_df['price']
bmodel = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        00:41:18   Log-Likelihood:                -95.143
No. Observations:                 205   AIC:                             220.3
Df Residuals:                     190   BIC:                             270.1
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.769e-16      0.028   6.34e-15      1.000      -0.055       0.055
symboling            0.0446      0.038      1.175      0.242      -0.030       0.119
wheelbase            0.1264      0.081      1.561      0.120      -0.033       0.286
carlength           -0.1464      0.086     -1.708      0.089      -0.315       0.023
carwidth             0.1253      0.067      1.882      0.061      -0.006       0.257
carheight            0.0596      0.042      1.409      0.160      -0.024       0.143
curbweight           0.1224      0.113      1.082      0.281      -0.101       0.346
enginesize           0.6087      0.072      8.443      0.000       0.467       0.751
boreratio           -0.0334      0.041     -0.824      0.411      -0.113       0.047
stroke              -0.1200      0.031     -3.928      0.000      -0.180      -0.060
compressionratio     0.1424      0.041      3.434      0.001       0.061       0.224
horsepower           0.1609      0.081      1.998      0.047       0.002       0.320
peakrpm              0.1408      0.040      3.518      0.001       0.062       0.220
citympg             -0.2350      0.147     -1.595      0.112      -0.526       0.056
highwaympg           0.1649      0.138      1.196      0.233      -0.107       0.437
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                         18.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('carlength', 'carwidth'): 0.84
('carwidth', 'enginesize'): 0.74
('enginesize', 'horsepower'): 0.81


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">ROBUST SCALER
</h1>

In [64]:
robust_df = df.copy()

In [65]:
robust_scaler = RobustScaler()
for col in num_cols:
    robust_df[col] = robust_scaler.fit_transform(df[[col]])


###### Robust dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değeriyle aynı çıktığı gözlemlendi

In [66]:
X = sm.add_constant(robust_df.drop("price",axis=1))
y = robust_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        00:41:18   Log-Likelihood:                -76.807
No. Observations:                 205   AIC:                             183.6
Df Residuals:                     190   BIC:                             233.5
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.1490      0.039      3.816      0.000       0.072       0.226
symboling            0.0656      0.056      1.175      0.242      -0.045       0.176
wheelbase            0.1520      0.097      1.561      0.120      -0.040       0.344
carlength           -0.1828      0.107     -1.708      0.089      -0.394       0.028
carwidth             0.1499      0.080      1.882      0.061      -0.007       0.307
carheight            0.0782      0.056      1.409      0.160      -0.031       0.188
curbweight           0.1702      0.157      1.082      0.281      -0.140       0.481
enginesize           0.5896      0.070      8.443      0.000       0.452       0.727
boreratio           -0.0486      0.059     -0.824      0.411      -0.165       0.068
stroke              -0.1052      0.027     -3.928      0.000      -0.158      -0.052
compressionratio     0.0263      0.008      3.434      0.001       0.011       0.041
horsepower           0.1716      0.086      1.998      0.047       0.002       0.341
peakrpm              0.1894      0.054      3.518      0.001       0.083       0.296
citympg             -0.3622      0.227     -1.595      0.112      -0.810       0.086
highwaympg           0.1976      0.165      1.196      0.233      -0.128       0.523
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                         56.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('wheelbase', 'curbweight'): 0.78
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.88
('carlength', 'highwaympg'): -0.70
('carwidth', 'curbweight'): 0.87
('carwidth', 'enginesize'): 0.74
('curbweight', 'enginesize'): 0.85
('curbweight', 'horsepower'): 0.75
('curbweight', 'citympg'): -0.76
('curbweight', 'highwaympg'): -0.80
('enginesize', 'horsepower'): 0.81
('horsepower', 'citympg'): -0.80
('horsepower', 'highwaympg'): -0.77
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 3'e düştüğü gözlemlendi.

In [68]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
13,citympg,27.824477
14,highwaympg,24.369050
6,curbweight,16.413566


In [69]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri Robust  dönüşümü ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi

In [70]:
X = sm.add_constant(X)
y = robust_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     98.03
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           8.80e-73
Time:                        00:41:18   Log-Likelihood:                -79.323
No. Observations:                 205   AIC:                             182.6
Df Residuals:                     193   BIC:                             222.5
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.1335      0.034      3.876      0.000       0.066       0.201
symboling            0.0901      0.054      1.654      0.100      -0.017       0.197
wheelbase            0.1570      0.094      1.665      0.098      -0.029       0.343
carlength           -0.0733      0.094     -0.776      0.439      -0.259       0.113
carwidth             0.1653      0.079      2.090      0.038       0.009       0.321
carheight            0.0783      0.056      1.408      0.161      -0.031       0.188
enginesize           0.5866      0.064      9.138      0.000       0.460       0.713
boreratio           -0.0340      0.059     -0.579      0.563      -0.150       0.082
stroke              -0.1005      0.027     -3.761      0.000      -0.153      -0.048
compressionratio     0.0238      0.006      3.828      0.000       0.012       0.036
horsepower           0.2654      0.074      3.581      0.000       0.119       0.412
peakrpm              0.1825      0.053      3.440      0.001       0.078       0.287
==============================================================================
Omnibus:                       23.011   Durbin-Watson:                   0.902
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.223
Skew:                           0.410   Prob(JB):                     6.87e-15
Kurtosis:                       5.639   Cond. No.                         24.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [71]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('carlength', 'carwidth'): 0.84
('carwidth', 'enginesize'): 0.74
('enginesize', 'horsepower'): 0.81


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF"> MaxAbsScaler
</h1>

In [72]:
max_abs_scaler_df = df.copy()

In [98]:
max_abs_scaler = MaxAbsScaler()
for col in num_cols:
    max_abs_scaler_df[col] = max_abs_scaler.fit_transform(df[[col]])


In [99]:
max_abs_scaler_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1.000000,0.732837,0.811148,0.886584,0.816054,0.626660,0.398773,0.880711,0.642686,0.391304,0.385417,0.757576,0.428571,0.500000,0.297247
1,1.000000,0.732837,0.811148,0.886584,0.816054,0.626660,0.398773,0.880711,0.642686,0.391304,0.385417,0.757576,0.428571,0.500000,0.363436
2,0.333333,0.781638,0.822681,0.905947,0.876254,0.694294,0.466258,0.680203,0.832134,0.391304,0.534722,0.757576,0.387755,0.481481,0.363436
3,0.666667,0.825476,0.848630,0.915629,0.908027,0.574766,0.334356,0.809645,0.815348,0.434783,0.354167,0.833333,0.489796,0.555556,0.307269
4,0.666667,0.822167,0.848630,0.918396,0.908027,0.694540,0.417178,0.809645,0.815348,0.347826,0.399306,0.833333,0.367347,0.407407,0.384361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-0.333333,0.902399,0.907256,0.952974,0.928094,0.726021,0.432515,0.959391,0.755396,0.413043,0.395833,0.818182,0.469388,0.518519,0.371035
201,-0.333333,0.902399,0.907256,0.951591,0.928094,0.749877,0.432515,0.959391,0.755396,0.378261,0.555556,0.803030,0.387755,0.462963,0.419493
202,-0.333333,0.902399,0.907256,0.952974,0.928094,0.740777,0.530675,0.908629,0.688249,0.382609,0.465278,0.833333,0.367347,0.425926,0.473238
203,-0.333333,0.902399,0.907256,0.952974,0.928094,0.791195,0.444785,0.763959,0.815348,1.000000,0.368056,0.727273,0.530612,0.500000,0.494934


###### MaxAbs dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değeriyle aynı çıktığı gözlemlendi.

In [100]:
X = sm.add_constant(max_abs_scaler_df.drop("price",axis=1))
y = max_abs_scaler_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     78.05
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.97e-71
Time:                        01:05:58   Log-Likelihood:                 261.54
No. Observations:                 205   AIC:                            -493.1
Df Residuals:                     190   BIC:                            -443.2
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -1.1377      0.345     -3.299      0.001      -1.818      -0.457
symboling            0.0189      0.016      1.175      0.242      -0.013       0.051
wheelbase            0.4466      0.286      1.561      0.120      -0.118       1.011
carlength           -0.4346      0.254     -1.708      0.089      -0.936       0.067
carwidth             0.7431      0.395      1.882      0.061      -0.036       1.522
carheight            0.2565      0.182      1.409      0.160      -0.103       0.616
curbweight           0.1682      0.155      1.082      0.281      -0.138       0.475
enginesize           0.8386      0.099      8.443      0.000       0.643       1.034
boreratio           -0.0854      0.104     -0.824      0.411      -0.290       0.119
stroke              -0.2807      0.071     -3.928      0.000      -0.422      -0.140
compressionratio     0.1451      0.042      3.434      0.001       0.062       0.228
horsepower           0.2062      0.103      1.998      0.047       0.003       0.410
peakrpm              0.3428      0.097      3.518      0.001       0.151       0.535
citympg             -0.3097      0.194     -1.595      0.112      -0.693       0.073
highwaympg           0.2275      0.190      1.196      0.233      -0.148       0.603
==============================================================================
Omnibus:                       24.845   Durbin-Watson:                   0.903
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.581
Skew:                           0.412   Prob(JB):                     8.64e-18
Kurtosis:                       5.919   Cond. No.                         279.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [76]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('wheelbase', 'curbweight'): 0.78
('carlength', 'carwidth'): 0.84
('carlength', 'curbweight'): 0.88
('carlength', 'highwaympg'): -0.70
('carwidth', 'curbweight'): 0.87
('carwidth', 'enginesize'): 0.74
('curbweight', 'enginesize'): 0.85
('curbweight', 'horsepower'): 0.75
('curbweight', 'citympg'): -0.76
('curbweight', 'highwaympg'): -0.80
('enginesize', 'horsepower'): 0.81
('horsepower', 'citympg'): -0.80
('horsepower', 'highwaympg'): -0.77
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 4'e düştüğü gözlemlendi.

In [77]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
13,citympg,27.824477
14,highwaympg,24.369050
6,curbweight,16.413566


In [78]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri MaxAbsScaler ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.

In [79]:
X = sm.add_constant(X)
y = max_abs_scaler_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.840
Method:                 Least Squares   F-statistic:                     98.03
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           8.80e-73
Time:                        00:41:18   Log-Likelihood:                -97.659
No. Observations:                 205   AIC:                             219.3
Df Residuals:                     193   BIC:                             259.2
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             1.769e-16      0.028   6.31e-15      1.000      -0.055       0.055
symboling            0.0612      0.037      1.654      0.100      -0.012       0.134
wheelbase            0.1305      0.078      1.665      0.098      -0.024       0.285
carlength           -0.0587      0.076     -0.776      0.439      -0.208       0.091
carwidth             0.1382      0.066      2.090      0.038       0.008       0.269
carheight            0.0596      0.042      1.408      0.161      -0.024       0.143
enginesize           0.6056      0.066      9.138      0.000       0.475       0.736
boreratio           -0.0234      0.040     -0.579      0.563      -0.103       0.056
stroke              -0.1146      0.030     -3.761      0.000      -0.175      -0.055
compressionratio     0.1289      0.034      3.828      0.000       0.062       0.195
horsepower           0.2489      0.070      3.581      0.000       0.112       0.386
peakrpm              0.1356      0.039      3.440      0.001       0.058       0.213
==============================================================================
Omnibus:                       23.011   Durbin-Watson:                   0.902
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               65.223
Skew:                           0.410   Prob(JB):                     6.87e-15
Kurtosis:                       5.639   Cond. No.                         7.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.80
('carlength', 'carwidth'): 0.84
('carwidth', 'enginesize'): 0.74
('enginesize', 'horsepower'): 0.81


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">QuantileTransformer
</h1>

In [81]:
quantile_transformer_df = df.copy()

In [82]:
quantile_transformer = QuantileTransformer()
for col in num_cols:
    quantile_transformer_df[col] = quantile_transformer.fit_transform(df[[col]])

In [83]:
quantile_transformer_df

,symboling,wheelbase,carlength,carwidth,carheight,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1.000000,0.017157,0.306373,0.252451,0.007353,0.580882,0.644608,0.678922,0.075980,0.446078,0.659314,0.401961,0.301471,0.318627,0.642157
1,1.000000,0.017157,0.306373,0.252451,0.007353,0.580882,0.644608,0.678922,0.075980,0.446078,0.659314,0.401961,0.301471,0.318627,0.742647
2,0.580882,0.235294,0.397059,0.507353,0.299020,0.705882,0.806373,0.004902,0.821078,0.446078,0.862745,0.401961,0.200980,0.299020,0.742647
3,0.794118,0.688725,0.644608,0.593137,0.556373,0.404412,0.406863,0.375000,0.693627,0.887255,0.593137,0.799020,0.453431,0.480392,0.671569
4,0.794118,0.676471,0.644608,0.627451,0.556373,0.710784,0.696078,0.375000,0.693627,0.110294,0.710784,0.799020,0.127451,0.061275,0.799020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,0.068627,0.928922,0.877451,0.914216,0.735294,0.754902,0.735294,0.963235,0.301471,0.821078,0.693627,0.678922,0.370098,0.362745,0.774510
201,0.068627,0.928922,0.877451,0.901961,0.735294,0.803922,0.735294,0.963235,0.301471,0.294118,0.899510,0.642157,0.200980,0.245098,0.867647
202,0.068627,0.928922,0.877451,0.914216,0.735294,0.774510,0.882353,0.737745,0.098039,0.323529,0.799020,0.799020,0.127451,0.100490,0.887255
203,0.068627,0.928922,0.877451,0.914216,0.735294,0.892157,0.754902,0.122549,0.693627,1.000000,0.607843,0.242647,0.575980,0.318627,0.897059


###### Quantile Transformer dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değerinden yüksek çıktı.

In [84]:
X = sm.add_constant(quantile_transformer_df.drop("price",axis=1))
y = quantile_transformer_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.890
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     110.2
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           3.94e-83
Time:                        00:41:19   Log-Likelihood:                 189.42
No. Observations:                 205   AIC:                            -348.8
Df Residuals:                     190   BIC:                            -299.0
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0156      0.091      0.173      0.863      -0.163       0.194
symboling            0.0405      0.033      1.230      0.220      -0.024       0.106
wheelbase            0.1295      0.077      1.683      0.094      -0.022       0.281
carlength           -0.0455      0.093     -0.491      0.624      -0.229       0.138
carwidth             0.0627      0.061      1.031      0.304      -0.057       0.183
carheight            0.0204      0.038      0.541      0.589      -0.054       0.095
curbweight           0.5052      0.094      5.397      0.000       0.321       0.690
enginesize           0.0406      0.071      0.571      0.569      -0.100       0.181
boreratio           -0.0706      0.040     -1.777      0.077      -0.149       0.008
stroke              -0.0590      0.031     -1.910      0.058      -0.120       0.002
compressionratio     0.1217      0.033      3.636      0.000       0.056       0.188
horsepower           0.3228      0.081      4.001      0.000       0.164       0.482
peakrpm              0.0112      0.038      0.296      0.768      -0.063       0.086
citympg             -0.3064      0.132     -2.323      0.021      -0.567      -0.046
highwaympg           0.1948      0.108      1.800      0.073      -0.019       0.408
==============================================================================
Omnibus:                        1.704   Durbin-Watson:                   0.987
Prob(Omnibus):                  0.427   Jarque-Bera (JB):                1.336
Skew:                           0.145   Prob(JB):                        0.513
Kurtosis:                       3.269   Cond. No.                         51.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.91
('wheelbase', 'carwidth'): 0.81
('wheelbase', 'curbweight'): 0.77
('carlength', 'carwidth'): 0.89
('carlength', 'curbweight'): 0.89
('carlength', 'enginesize'): 0.78
('carwidth', 'curbweight'): 0.86
('carwidth', 'enginesize'): 0.77
('carwidth', 'highwaympg'): -0.70
('curbweight', 'enginesize'): 0.88
('curbweight', 'boreratio'): 0.70
('curbweight', 'horsepower'): 0.81
('curbweight', 'citympg'): -0.81
('curbweight', 'highwaympg'): -0.83
('enginesize', 'boreratio'): 0.70
('enginesize', 'horsepower'): 0.82
('enginesize', 'citympg'): -0.73
('enginesize', 'highwaympg'): -0.72
('horsepower', 'citympg'): -0.91
('horsepower', 'highwaympg'): -0.89
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayımız 7'ye düştü

In [86]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
0,const,168.895546
13,citympg,29.950242
14,highwaympg,20.202686
6,curbweight,15.190571
3,carlength,14.916231
11,horsepower,11.273471
2,wheelbase,10.248839


In [87]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri QuantileTransformer  ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktı

In [88]:
X = sm.add_constant(X)
y = quantile_transformer_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.799
Method:                 Least Squares   F-statistic:                     102.1
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           1.08e-65
Time:                        00:41:19   Log-Likelihood:                 131.19
No. Observations:                 205   AIC:                            -244.4
Df Residuals:                     196   BIC:                            -214.5
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const               -0.0865      0.052     -1.670      0.097      -0.189       0.016
symboling            0.0463      0.037      1.254      0.211      -0.026       0.119
carwidth             0.4150      0.053      7.763      0.000       0.310       0.520
carheight            0.0505      0.040      1.278      0.203      -0.027       0.128
enginesize           0.5804      0.061      9.510      0.000       0.460       0.701
boreratio            0.0244      0.050      0.484      0.629      -0.075       0.124
stroke              -0.1389      0.037     -3.778      0.000      -0.211      -0.066
compressionratio     0.0203      0.033      0.620      0.536      -0.044       0.085
peakrpm              0.1743      0.035      4.934      0.000       0.105       0.244
==============================================================================
Omnibus:                        6.281   Durbin-Watson:                   1.098
Prob(Omnibus):                  0.043   Jarque-Bera (JB):                6.159
Skew:                           0.423   Prob(JB):                       0.0460
Kurtosis:                       3.079   Cond. No.                         14.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [89]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('carwidth', 'enginesize'): 0.77
('enginesize', 'boreratio'): 0.70


<h1 style="background-color:orange;font-family:newtimeroman;font-size:175%;text-align:left;color:#FFFFFF">PowerTransformer
</h1>

In [90]:
power_transformer_df = df.copy()

In [91]:
power_transformer = PowerTransformer()
for col in num_cols:
    power_transformer_df[col] = power_transformer.fit_transform(df[[col]])

###### Power Transformer dönüşümü yaptıktan sonra $R^2$ değeri orijinal veri setinin $R^2$ değerinden yüksek çıktığı gözlemlendi.


In [92]:
X = sm.add_constant(power_transformer_df.drop("price",axis=1))
y = power_transformer_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.880
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     107.9
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           1.66e-80
Time:                        00:41:19   Log-Likelihood:                -73.434
No. Observations:                 205   AIC:                             174.9
Df Residuals:                     191   BIC:                             221.4
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const             7.149e-08      0.025   2.85e-06      1.000      -0.049       0.049
symboling            0.0152      0.035      0.439      0.661      -0.053       0.083
wheelbase           -0.0053      0.064     -0.083      0.934      -0.131       0.120
carlength            0.0562      0.079      0.708      0.480      -0.100       0.213
carwidth           3.45e-17   3.33e-17      1.037      0.301   -3.11e-17       1e-16
carheight            0.0221      0.037      0.589      0.557      -0.052       0.096
curbweight           0.4867      0.099      4.936      0.000       0.292       0.681
enginesize           0.0201      0.063      0.317      0.752      -0.105       0.145
boreratio           -0.0390      0.037     -1.043      0.298      -0.113       0.035
stroke              -0.0177      0.029     -0.613      0.541      -0.075       0.039
compressionratio     0.1655      0.035      4.711      0.000       0.096       0.235
horsepower           0.3608      0.086      4.215      0.000       0.192       0.530
peakrpm              0.0039      0.038      0.104      0.918      -0.071       0.078
citympg             -0.3866      0.127     -3.045      0.003      -0.637      -0.136
highwaympg           0.2486      0.109      2.284      0.023       0.034       0.463
==============================================================================
Omnibus:                        0.560   Durbin-Watson:                   1.085
Prob(Omnibus):                  0.756   Jarque-Bera (JB):                0.297
Skew:                           0.052   Prob(JB):                        0.862
Kurtosis:                       3.154   Cond. No.                     7.63e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.19e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [93]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carlength'): 0.87
('wheelbase', 'carwidth'): 0.77
('wheelbase', 'curbweight'): 0.74
('carlength', 'carwidth'): 0.85
('carlength', 'curbweight'): 0.89
('carlength', 'enginesize'): 0.75
('carlength', 'highwaympg'): -0.71
('carwidth', 'curbweight'): 0.86
('carwidth', 'enginesize'): 0.75
('carwidth', 'highwaympg'): -0.71
('curbweight', 'enginesize'): 0.86
('curbweight', 'horsepower'): 0.80
('curbweight', 'citympg'): -0.81
('curbweight', 'highwaympg'): -0.84
('enginesize', 'horsepower'): 0.79
('enginesize', 'highwaympg'): -0.71
('horsepower', 'citympg'): -0.91
('horsepower', 'highwaympg'): -0.88
('citympg', 'highwaympg'): 0.97


###### Çoklu bağlantı problemine sahip değişken sayısının 5'e düştüğü gözlemlendi.

In [94]:
df_vif_data = pd.DataFrame()
df_vif_data["Variable"] = X.columns
df_vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
df_vif_data[df_vif_data["VIF"]>10].sort_values(ascending=False,by="VIF")

,Variable,VIF
13,citympg,25.691512
14,highwaympg,18.868854
6,curbweight,15.491245
11,horsepower,11.678781
3,carlength,10.058789


In [95]:
remove_variable = df_vif_data[df_vif_data["VIF"] > 10]["Variable"].tolist()
X.drop(remove_variable,axis=1,inplace=True)

######  Çoklu bağlantı problemi olan değişkenleri kaldırdıktan sonra $R^2$ değeri PowerTransformer   ile dönüştürülmüş veri setinin $R^2$ değerinden düşük çıktığı gözlemlendi.


In [96]:
X = sm.add_constant(X)
y = power_transformer_df['price']
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     75.77
Date:                Wed, 24 Jan 2024   Prob (F-statistic):           7.68e-56
Time:                        00:41:19   Log-Likelihood:                -146.44
No. Observations:                 205   AIC:                             310.9
Df Residuals:                     196   BIC:                             340.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -5.173e-06      0.035     -0.000      1.000      -0.070       0.070
symboling            0.1477      0.046      3.239      0.001       0.058       0.238
wheelbase            0.3811      0.061      6.239      0.000       0.261       0.502
carwidth          6.279e-16   5.41e-17     11.597      0.000    5.21e-16    7.35e-16
carheight           -0.0426      0.049     -0.861      0.390      -0.140       0.055
enginesize           0.6373      0.055     11.661      0.000       0.530       0.745
boreratio            0.1170      0.050      2.348      0.020       0.019       0.215
stroke              -0.0770      0.039     -1.966      0.051      -0.154       0.000
compressionratio     0.0624      0.038      1.635      0.104      -0.013       0.138
peakrpm              0.2193      0.041      5.408      0.000       0.139       0.299
==============================================================================
Omnibus:                        5.010   Durbin-Watson:                   1.090
Prob(Omnibus):                  0.082   Jarque-Bera (JB):                5.079
Skew:                           0.252   Prob(JB):                       0.0789
Kurtosis:                       3.583   Cond. No.                     3.93e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.89e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [97]:
correlation_matrix = X.corr()

high_correlation_pairs = []

for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
            high_correlation_pairs.append(pair)
print("Korelasyonları 0.7'den büyük olan değişken çiftleri:")

for pair in high_correlation_pairs:
    i, j = correlation_matrix.columns.get_loc(pair[0]), correlation_matrix.columns.get_loc(pair[1])
    correlation_coefficient = correlation_matrix.iloc[i, j]
    print(f"{pair}: {correlation_coefficient:.2f}")

Korelasyonları 0.7'den büyük olan değişken çiftleri:
('wheelbase', 'carwidth'): 0.77
('carwidth', 'enginesize'): 0.75
